In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [10]:
from Transformer_RPN.logger import logging

In [13]:
logger = logging.getLogger(__name__)

In [14]:
logger.info("Checking logger setup")

# Data Loaders

In [4]:
import os
import tqdm

In [3]:
os.chdir('../')

In [3]:
list = [os.path.join('Data/Strawberry data/strawberry_labels', dir, i) for dir in os.listdir('Data/Strawberry data/strawberry_labels') for i in os.listdir(os.path.join('Data/Strawberry data/strawberry_labels', dir))]

In [4]:
list

['Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot192.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot94.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot225.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot83.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot134.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot99.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot213.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot151.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot24.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot198.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot155.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot194.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot59.xml',
 'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot54.xml',
 'Data/Strawberry data/str

In [5]:
img_info = {}

In [3]:
import xml.etree.ElementTree as ET

In [7]:
list[0]

'Data/Strawberry data/strawberry_labels/leaf_spot/leaf_spot192.xml'

In [ ]:
img_info['img_id'] = os.path.basename(list[0]).split('.xml')[0]

In [9]:
img_info['img_id']

'leaf_spot192'

In [10]:
img_info['filename'] = os.path.join('Data/Strawberry data/strawberry_data'+'/leaf_spot', '{}.jpg'.format(img_info['img_id']))

In [11]:
img_info['filename']

'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot192.jpg'

In [12]:
anotation_info = ET.parse(list[0])

In [13]:
root = anotation_info.getroot()

In [14]:
size = root.find('size')

In [15]:
size

<Element 'size' at 0x7489d44559e0>

In [16]:
width = int(size.find('width').text)
height = int(size.find('height').text)

In [33]:
width

269

In [34]:
height

282

In [17]:
img_info['width'] = width
img_info['height'] = height

In [18]:
img_info

{'img_id': 'leaf_spot192',
 'filename': 'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot192.jpg',
 'width': 269,
 'height': 282}

In [19]:
detections = []

In [20]:
obj = anotation_info.findall('object')

In [21]:
label = obj[0].find('name').text

In [22]:
classes = ['angular_leafspot', 'anthracnose_fruit_rot', 'gray_mold', 'leaf_scorch', 'leaf_spot']

In [23]:
label2idx = {classes[idx]: idx for idx in range(len(classes))}

In [24]:
label2idx

{'angular_leafspot': 0,
 'anthracnose_fruit_rot': 1,
 'gray_mold': 2,
 'leaf_scorch': 3,
 'leaf_spot': 4}

In [25]:
label = label2idx[label]

In [26]:
boundingbox_info = obj[0].find('bndbox')

In [27]:
bbox = [
                int(float(boundingbox_info.find('xmin').text))-1,
                int(float(boundingbox_info.find('ymin').text))-1,
                int(float(boundingbox_info.find('xmax').text))-1,
                int(float(boundingbox_info.find('ymax').text))-1
            ]

In [28]:
bbox

[37, 0, 63, 22]

In [29]:
det = {}

In [30]:
det['label'] = label
det['bbox'] = bbox

In [31]:
det

{'label': 4, 'bbox': [37, 0, 63, 22]}

In [33]:
detections

[{'label': 4, 'bbox': [37, 0, 63, 22]}]

In [1]:
from PIL import Image

In [5]:
from tqdm import tqdm

In [6]:
class LoadData:
    def __init__(self, img_dir, xml_dir, label2idx):
        self.img_dir = img_dir
        self.xml_dir = xml_dir
        self.label2idx = label2idx
        self.img_infos = []

    def load(self):
        xml_files = [os.path.join(self.xml_dir, dir, file) for dir in os.listdir(self.xml_dir) for file in os.listdir(os.path.join(self.xml_dir, dir))]
        for file in tqdm(xml_files, desc='Processing XML files'):
            img_info = {}
            img_info['id'] = os.path.basename(file).split('.xml')[0]
            xml_info = ET.parse(file)
            root = xml_info.getroot()
            size = root.find('size')
            folder = file.split('/')[3]
            img_info['image'] = os.path.join(self.img_dir, folder,'{}.jpg'.format(img_info['id']))
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            img_info['height'] = height
            img_info['width'] = width
            detections = []
        

            for obj in xml_info.findall('object'):
                det = {}
                label = label2idx[obj.find('name').text]
                if obj.find('name').text == 'leaf_blight':
                    print(img_info['id'])
                bbox_info = obj.find('bndbox')
                bbox = [
                    int(float(bbox_info.find('xmin').text))-1,
                    int(float(bbox_info.find('ymin').text))-1,
                    int(float(bbox_info.find('xmax').text))-1,
                    int(float(bbox_info.find('ymax').text))-1
                ]
                det['label'] = label
                det['bbox'] = bbox
                detections.append(det)
            
            img_info['detections'] = detections
            self.img_infos.append(img_info)
        return self.img_infos

In [7]:
img_dir = r'Data/Strawberry data/strawberry_data'
xml_dir = r'Data/Strawberry data/strawberry_labels'
classes = ['angular_leafspot', 'anthracnose_fruit_rot', 'gray_mold', 'leaf_scorch', 'leaf_spot']
classes = sorted(classes)
classes = ['background'] + classes
label2idx = {classes[idx]: idx for idx in range(len(classes))}
idx2label = {idx: classes[idx] for idx in range(len(classes))}


In [48]:
classes

['background',
 'angular_leafspot',
 'anthracnose_fruit_rot',
 'gray_mold',
 'leaf_scorch',
 'leaf_spot']

In [41]:
label2idx

{'background': 0,
 'angular_leafspot': 1,
 'anthracnose_fruit_rot': 2,
 'gray_mold': 3,
 'leaf_scorch': 4,
 'leaf_spot': 5}

In [9]:
os.chdir('../')

In [10]:
ld = LoadData(img_dir=img_dir, xml_dir=xml_dir, label2idx=label2idx)
img_infos = ld.load()

Processing XML files:   0%|          | 0/941 [00:00<?, ?it/s]

Processing XML files: 100%|██████████| 941/941 [00:00<00:00, 10185.50it/s]


In [14]:
img_infos

[{'id': 'leaf_spot192',
  'image': 'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot192.jpg',
  'height': 282,
  'width': 269,
  'detections': [{'label': 5, 'bbox': [37, 0, 63, 22]},
   {'label': 5, 'bbox': [33, 29, 61, 50]},
   {'label': 5, 'bbox': [22, 79, 43, 98]},
   {'label': 5, 'bbox': [108, 64, 134, 93]},
   {'label': 5, 'bbox': [66, 86, 123, 125]},
   {'label': 5, 'bbox': [26, 106, 76, 133]},
   {'label': 5, 'bbox': [42, 160, 74, 185]},
   {'label': 5, 'bbox': [73, 144, 107, 172]},
   {'label': 5, 'bbox': [98, 130, 128, 156]},
   {'label': 5, 'bbox': [167, 103, 188, 136]},
   {'label': 5, 'bbox': [199, 139, 221, 161]},
   {'label': 5, 'bbox': [83, 187, 134, 226]},
   {'label': 5, 'bbox': [138, 189, 176, 222]},
   {'label': 5, 'bbox': [197, 198, 235, 224]},
   {'label': 4, 'bbox': [137, 42, 153, 60]},
   {'label': 4, 'bbox': [161, 26, 168, 35]}]},
 {'id': 'leaf_spot94',
  'image': 'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot94.jpg',
  'height': 233,
  'width

In [15]:
img_infos

[{'id': 'leaf_spot192',
  'image': 'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot192.jpg',
  'height': 282,
  'width': 269,
  'detections': [{'label': 5, 'bbox': [37, 0, 63, 22]},
   {'label': 5, 'bbox': [33, 29, 61, 50]},
   {'label': 5, 'bbox': [22, 79, 43, 98]},
   {'label': 5, 'bbox': [108, 64, 134, 93]},
   {'label': 5, 'bbox': [66, 86, 123, 125]},
   {'label': 5, 'bbox': [26, 106, 76, 133]},
   {'label': 5, 'bbox': [42, 160, 74, 185]},
   {'label': 5, 'bbox': [73, 144, 107, 172]},
   {'label': 5, 'bbox': [98, 130, 128, 156]},
   {'label': 5, 'bbox': [167, 103, 188, 136]},
   {'label': 5, 'bbox': [199, 139, 221, 161]},
   {'label': 5, 'bbox': [83, 187, 134, 226]},
   {'label': 5, 'bbox': [138, 189, 176, 222]},
   {'label': 5, 'bbox': [197, 198, 235, 224]},
   {'label': 4, 'bbox': [137, 42, 153, 60]},
   {'label': 4, 'bbox': [161, 26, 168, 35]}]},
 {'id': 'leaf_spot94',
  'image': 'Data/Strawberry data/strawberry_data/leaf_spot/leaf_spot94.jpg',
  'height': 233,
  'width

In [11]:
from torch.utils.data.dataset import Dataset

In [12]:
import random

In [13]:
import torchvision

In [14]:
import torch

In [36]:
class CustomDataset(Dataset):
    def __init__(self, split, img_dir, xml_dir, label2idx, target_size=(224, 224)):
        self.split = split
        self.img_dir = img_dir
        self.xml_dir = xml_dir
        self.target_size = target_size  # Resize target (224, 224)
        self.images_info = LoadData(self.img_dir, self.xml_dir, label2idx).load()
        self.transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(self.target_size),  # Resize images
            torchvision.transforms.ToTensor()  # Convert to tensor
        ])
    
    def __len__(self):
        return len(self.images_info)
    
    def __getitem__(self, index):
        img_info = self.images_info[index]
        img = Image.open(img_info['image'])
        
        # Store original dimensions
        original_w, original_h = img.size
        target_w, target_h = self.target_size
        
        # Random horizontal flip (for training only)
        to_flip = False
        if self.split == 'train' and random.random() < 0.5:
            to_flip = True
            img = img.transpose(Image.FLIP_LEFT_RIGHT)

        # Resize the image
        img_tensor = self.transform(img)
        
        # Scale bounding boxes to new image size
        targets = {}
        targets['bboxes'] = []
        targets['labels'] = torch.as_tensor([d['label'] for d in img_info['detections']], dtype=torch.int64)

        scale_x = target_w / original_w
        scale_y = target_h / original_h

        for detection in img_info['detections']:
            x1, y1, x2, y2 = detection['bbox']

            # Flip bounding box if image is flipped
            if to_flip:
                x1, x2 = original_w - x2, original_w - x1

            # Scale bbox to new dimensions
            x1 = int(x1 * scale_x)
            y1 = int(y1 * scale_y)
            x2 = int(x2 * scale_x)
            y2 = int(y2 * scale_y)

            targets['bboxes'].append([x1, y1, x2, y2])

        targets['bboxes'] = torch.as_tensor(targets['bboxes'], dtype=torch.float32)

        return img_tensor, targets, img_info['image']

In [37]:
ds = CustomDataset('train', img_dir, xml_dir, label2idx)

Processing XML files: 100%|██████████| 941/941 [00:00<00:00, 6084.06it/s]


In [38]:
ds[4][0].shape

torch.Size([3, 224, 224])

In [39]:
ds[5][0].shape

torch.Size([3, 224, 224])

In [40]:
from torch.utils.data.dataloader import DataLoader

In [44]:
train_dataset = DataLoader(ds,batch_size=1,shuffle=True,num_workers=4)

In [41]:
train_dataset.dataset.__sizeof__

<function CustomDataset.__sizeof__()>

In [48]:
for image, target, _ in tqdm(train_dataset):
    print(target['bboxes'])
    print(image.shape)
    image = image
    break
    


  0%|          | 0/941 [00:00<?, ?it/s]

tensor([[[ 79.,  51., 152., 144.]]])
torch.Size([1, 3, 224, 224])


In [51]:
image[0].shape

torch.Size([3, 224, 224])

In [63]:
import numpy as np

In [93]:
def sliding_window_patches(image_tensor, patch_size=(16, 16), stride=(16, 16)):
    _, h, w = image_tensor.shape
    ph, pw = patch_size
    stride_h, stride_w = stride

    # Collect patches
    patches = [
        image_tensor[:, i:i + ph, j:j + pw]
        for i in range(0, h - ph + 1, stride_h)
        for j in range(0, w - pw + 1, stride_w)
    ]

    return torch.stack(patches)  # Shape: (num_patches, C, ph, pw)


In [94]:
patches = sliding_window_patches(image_tensor=image[0])

In [95]:
patches[2].shape

torch.Size([3, 16, 16])

In [97]:
patches.shape

torch.Size([196, 3, 16, 16])

In [98]:
x_c = patches.reshape(-1, 3*16*16)

In [100]:
x_c.shape

torch.Size([196, 768])